## Import Package dan Panggil Data

In [ ]:
import os
import random
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error
)


In [ ]:
# SET SEED UNTUK REPRODUCIBILITY
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)

# Load Dataset (ambil hanya kolom Date dan Price)
df = pd.read_csv('/kaggle/input/skripsi/ihsg.csv', usecols=['Date', 'Price'])

# Format Date dan ubah Price menjadi float
df['Date'] = pd.to_datetime(df['Date'])
df['Price'] = df['Price'].replace({',': ''}, regex=True).astype('float64')

# Urutkan berdasarkan tanggal dan reset index
df.sort_values('Date', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

## Cek Data Kosong

In [ ]:
# Check Duplicates & Missing Values
print("Duplicated rows:", df.duplicated().sum())
print("Missing values:", df.isnull().sum().sum())

## Plot Harga

In [ ]:
# Plot Price History
fig = px.line(y=df.Price, x=df.Date)
fig.update_traces(line_color='black')
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price",
    title={'text': "IHSG History Data", 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(255,223,0,0.8)'
)

## Pembagian Data Train dan Data Test

In [ ]:
from datetime import datetime

# Split Train/Test 
split_date = datetime(2023, 4, 1)
test_size = df[df.Date >= split_date].shape[0]

In [ ]:
# Visualize Train/Test Split
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'yellow'
plt.rc('axes', edgecolor='white')
plt.plot(df.Date[:-test_size], df.Price[:-test_size], color='black', lw=2)
plt.plot(df.Date[-test_size:], df.Price[-test_size:], color='blue', lw=2)
plt.title('IHSG Training and Test Sets', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.legend(['Training set', 'Test set'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

## Normalisasi Data

In [ ]:
# Normalize Prices
scaler = MinMaxScaler()
scaler.fit(df.Price.values.reshape(-1, 1))

## Pemilihan Window Size

In [ ]:
# Window Size
window_size = 60

## Persiapan Data Train

In [ ]:
# Prepare Train Data
train_data = df.Price[:-test_size]
train_data_scaled = scaler.transform(train_data.values.reshape(-1, 1))

X_train, y_train = [], []
for i in range(window_size, len(train_data_scaled)):
    X_train.append(train_data_scaled[i - window_size:i, 0])
    y_train.append(train_data_scaled[i, 0])

In [ ]:
# Prepare Test Data
test_data = df.Price[-test_size - window_size:]
test_data_scaled = scaler.transform(test_data.values.reshape(-1, 1))

X_test, y_test = [], []
for i in range(window_size, len(test_data_scaled)):
    X_test.append(test_data_scaled[i - window_size:i, 0])
    y_test.append(test_data_scaled[i, 0])

In [ ]:
# Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

print('X_train Shape:', X_train.shape)
print('y_train Shape:', y_train.shape)
print('X_test Shape:', X_test.shape)
print('y_test Shape:', y_test.shape)

# Jalankan model secara bergantian agar mendapatkan output yang berbeda

## Model LSTM

### Kombinasi 1

In [ ]:
# Define LSTM (32,32) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = LSTM(units=32, return_sequences=True)(input1)
    x = Dropout(0.2)(x)
    x = LSTM(units=32)(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model


### Kombinasi 2

In [ ]:
# Define LSTM (64,32) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = LSTM(units=64, return_sequences=True)(input1)
    x = Dropout(0.2)(x)
    x = LSTM(units=32)(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model


### Kombinasi 3

In [ ]:
# Define LSTM (64,64) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = LSTM(units=64, return_sequences=True)(input1)
    x = Dropout(0.2)(x)
    x = LSTM(units=64)(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model

## Model GRU

### Kombinasi 1

In [ ]:
# Define GRU (32,32) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = GRU(units=32, return_sequences=True)(input1)  
    x = Dropout(0.2)(x)
    x = GRU(units=32)(x)                             
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model

### Kombinasi 2

In [ ]:
# Define GRU (64,32) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = GRU(units=64, return_sequences=True)(input1)  
    x = Dropout(0.2)(x)
    x = GRU(units=32)(x)                           
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model

### Kombinasi 3

In [ ]:
# Define GRU (64,64) Model
def define_model():
    input1 = Input(shape=(window_size, 1))
    x = GRU(units=64, return_sequences=True)(input1)  
    x = Dropout(0.2)(x)
    x = GRU(units=64)(x)                              
    x = Dropout(0.2)(x)
    x = Dense(32, activation='linear')(x)
    output = Dense(1)(x)

    model = Model(inputs=input1, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='Adam')
    model.summary()
    return model

In [ ]:
# Train Model
model = define_model()

## Pelatihan Data

## Untuk batch size 64

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

## Untuk batch size 32

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

## Evaluasi Data

In [ ]:
result = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

# Catat nilai MAPE agar dapat membandingkan outputnya

## Pengecekan nilai MAPE

In [ ]:
MAPE = mean_absolute_percentage_error(y_test, y_pred)

print("Test Loss:", result)
print("Test MAPE:", MAPE)

In [ ]:
# Inverse Transform
y_test_true = scaler.inverse_transform(y_test)
y_test_pred = scaler.inverse_transform(y_pred)

## Plot Perbandingan Hasil Prediksi vs Data Asli

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes', edgecolor='black')

plt.plot(df['Date'].iloc[-test_size:], y_test_true, color='blue', lw=2, label='Actual Test Data')
plt.plot(df['Date'].iloc[-test_size:], y_test_pred, color='red', lw=2, label='Predicted Test Data')

plt.title('Perbandingan Prediksi vs Aktual IHSG (Data Uji)', fontsize=15)
plt.xlabel('Tanggal', fontsize=12)
plt.ylabel('Harga Penutupan (IHSG)', fontsize=12)
plt.legend(loc='upper left', prop={'size': 12})
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()